In [ ]:
import pandas as pd
import numpy as np
import requests
import json

In [12]:
def IsCotractAdderss(Address):
    url="https://api.etherscan.io/api?module=contract&action=getabi&address="+Address+"&apikey=ZVF3M7DCCANE4MM82R79CQKMD728B8VWWA"
    try:
        response = requests.get(url)
    except Exception as e:
        return e
    
    data=response.json()

    if data.get('message')=="OK":
        return True
    else:
        return False
        
def EtherBalance(Address):
    apikey="ZVF3M7DCCANE4MM82R79CQKMD728B8VWWA"
    address=Address
    url="https://api.etherscan.io/api"+"?module=account"+"&action=balance"+"&address="+address+"&tag=latest"+"&apikey="+apikey
    
    try:
        response = requests.get(url)
    except Exception as e:
        return e
   
    data=response.json().get("result")
    
    return data

def NonmalTransection(Address):
    apikey="ZVF3M7DCCANE4MM82R79CQKMD728B8VWWA"
    address=Address
    
    
    url="https://api.etherscan.io/api?module=account&action=txlist&address="+address+"&startblock=0&endblock=99999999&page=1&offset=10000&sort=asc&apikey="+apikey
    
    try:
        response = requests.get(url)
    except Exception as e:
        return e
   
    data=response.json()

    t_list=data.get('result') # All transecton of the address

    # ****WE calculate the error free transection where "isError = 0" 
    # ****ALL the features are calculate based on valid transection. We discared the tracnection with error 

    Sent_tnx=0 # number of sent transections 
    Rec_tnx=0 # number of received transections 
    NumberofCreated_Contracts=0 # number of created contracts trnasections 
    TotalTransactions=0 # total number of transection 

    AvgMinBetweenSentTnx=0 # Average time between sent transection 
    AvgMinBetweenRecTnx=0 # Average time between received transection 
    TimeDiffBetweenFirstLastTnx=0 # Time difference between first and last transection 

    UniqueRecFromAddresses=0 # Total Unique addresses from which the account received transaction 
    UniqueSentFromAddresses = 0 # Total Unique addresses to which the account sent transaction 

    MinValueReceived=float('inf')  # Minimum value in Ether ever received 
    MaxValueReceived=float('-inf') # Maximum value in Ether ever received 
    AvgValueReceived=0 # Average value in Ether ever received 
    TotalEtherReceived=0 # Total Ether received for account address 

    MinValSent=float('inf') # Minimum value in Ether ever sent 
    MaxValSent=float('-inf') # Maximum value in Ether ever sent 
    AvgValSent=0 # Average value in Ether ever sent
    TotalEtherSent=0 # Total Ether received for account address

    MinValueSentToContract=float('inf') # Minimum value of Ether sent to a contract
    MaxValueSentToContract=float('-inf') # Maximum value of Ether sent to a contract
    AvgValueSentToContract=0 # Average value of Ether sent to contracts
    TotalEtherSent_Contracts=0 # Total value of Ether sent to contracts
    TotalTransactions=len(t_list)

    if TotalTransactions>0:   
        s_t_list=[]# list sent transection
        r_t_list=[]# list received transection
        c_t_list=[]# list contact transection
        s_adr=set() # SENT ADDRESS
        r_adr=set() # RECEIVED ADDRESS

        for i in t_list:
            if i["isError"]=="0":   # check if the transection is valid
                if i['to']=="" : # CONTRACT TRANSECTION BLOCK
                    c_t_list.append(i)
                    MinValueSentToContract = min(MinValueSentToContract, int(i['value']))# MINIMUM ETHER SENT TO CONTACTS 
                    MaxValueSentToContract = max(MaxValueSentToContract, int(i['value'])) # MAXIMUM ETHER SENT TO CONTACTS  
                    TotalEtherSent_Contracts = +int(i['value']) # TOTAL ETHER SENT TO CONTACTS

                elif i['from']==address : # SENT TRANSECTION BLOCK
                    s_t_list.append(i)
                    s_adr.add(i['to'])
                    MinValSent = min(MinValSent,int(i['value']))# MINIMUM ETHER SENT 
                    MaxValSent = max(MaxValSent,int(i['value'])) # MAXIMUM ETHER SENT 
                    TotalEtherSent = + int(i['value']) # TOTAL ETHER SENT 
                
                else: # RECEIVED TRANSECTION BLOCK
                    r_t_list.append(i)
                    r_adr.add(i['from'])
                    # print(i['from'])
                    MinValueReceived = min(MinValueReceived, int(i['value']))
                    MaxValueReceived = max(MaxValueReceived, int(i['value'])) 
                    TotalEtherReceived = + int(i['value']) 

        Sent_tnx = len(s_t_list) 
        Rec_tnx = len(r_t_list) 
        NumberofCreated_Contracts = len(c_t_list)
        
        if NumberofCreated_Contracts != 0:
            AvgValueSentToContract = TotalEtherSent_Contracts / NumberofCreated_Contracts 
        if Sent_tnx != 0:
            AvgValSent = TotalEtherSent / Sent_tnx 
        if Rec_tnx != 0:
            AvgValueReceived = TotalEtherReceived / Rec_tnx 
        UniqueSentFromAddresses = len(s_adr) 
        UniqueRecFromAddresses = len(r_adr) 
        
        if len(t_list)>1:
            TimeDiffBetweenFirstLastTnx=abs(int(t_list[0]['timeStamp'])-int(t_list[-1]['timeStamp']))
        if Sent_tnx>=2:
            for i in range(1,len(s_t_list)):
                AvgMinBetweenSentTnx+=abs( int(s_t_list[i]['timeStamp'])-int(s_t_list[i-1]['timeStamp']) )
            AvgMinBetweenSentTnx=(AvgMinBetweenSentTnx/(Sent_tnx-1))
        if Rec_tnx>=2:
            for i in range(1,len(r_t_list)):
                AvgMinBetweenRecTnx+=abs( int(r_t_list[i]['timeStamp'])-int(r_t_list[i-1]['timeStamp']) )
            AvgMinBetweenRecTnx=(AvgMinBetweenRecTnx/(Rec_tnx-1))
    
    if Sent_tnx==0:
        MinValSent=0
        MaxValSent=0
    if Rec_tnx==0:
        MinValueReceived=0
        MaxValueReceived=0
    if NumberofCreated_Contracts==0:
        MinValueSentToContract=0
        MaxValueSentToContract=0

    TotalEtherBalance=EtherBalance(Address)

    

    return {
        "Avg min between sent tnx": AvgMinBetweenSentTnx,
        "Avg min between received tnx": AvgMinBetweenRecTnx,
        "Time Diff between first and_last (Mins)": TimeDiffBetweenFirstLastTnx,
        "Sent_tnx": Sent_tnx,
        "Received_tnx": Rec_tnx,
        "NumberofCreated_Contracts": NumberofCreated_Contracts,
        "UniqueReceivedFrom_Addresses": UniqueRecFromAddresses,
        "UniqueSentTo_Addresses20": UniqueSentFromAddresses,
        "MinValueReceived": MinValueReceived,
        "MaxValueReceived": MaxValueReceived,
        "AvgValueReceived5Average": AvgValueReceived,
        "MinValSent": MinValSent,
        "MaxValSent": MaxValSent,
        "AvgValSent": AvgValSent,
        "MinValueSentToContract": MinValueSentToContract,
        "MaxValueSentToContract": MaxValueSentToContract,
        "AvgValueSentToContract": AvgValueSentToContract,
        "TotalTransactions(IncludingTnxtoCreate_Contract)": TotalTransactions,
        "TotalEtherSent": TotalEtherSent,
        "TotalEtherReceived": TotalEtherReceived,
        "TotalEtherSent_Contracts": TotalEtherSent_Contracts,
        "TotalEtherBalance": TotalEtherBalance
    }


def ERC20Transection(Address):

    apikey="ZVF3M7DCCANE4MM82R79CQKMD728B8VWWA"
    address=Address
    url="https://api.etherscan.io/api?module=account&action=tokentx&address="+address+"&page=1&offset=1000&startblock=0&endblock=27025780&sort=asc&apikey="+apikey

    try:
        response = requests.get(url)
    except Exception as e:
        return e

    data=response.json()

    TotalERC20Tnxs = 0 # Total number of ERC20 token transfer transactions

    ERC20TotalEther_Sent=0 # Total ERC20token sent transactions in Ether
    ERC20TotalEther_Received=0 # Total ERC20 token received in Ether
    ERC20TotalEtherSentContract=0 # Total ERC20 token transfer to other contracts in Ether
    # (** The total number of transection sent to any contract or recived from any contract **)

    ERC20UniqSent_Addr = 0 # Number of ERC20 token transactions sent to Unique account addresses
    ERC20UniqRec_Addr = 0 # Number of ERC20 token transactions received from Unique addresses
    ERC20UniqRecContractAddr = 0 # Number of ERC20token transactions received from Unique contract addresses
    # (** The total number of Contract addresses from, transection sent to any contract or recived from any contract **)

    ERC20AvgTimeBetweenSent_Tnx = 0 # Average time between ERC20 token sent transactions in minutes
    ERC20AvgTimeBetweenRec_Tnx = 0 # Average time between ERC20 token received transactions in minutes
    ERC20AvgTimeBetweenContract_Tnx=0 # Average time ERC20 token between sent token transactions
    # (** Average time between the transection realted to any contract address **)

    ERC20MinVal_Sent=float('inf') # Minimum value in Ether received from ERC20 token transactions for account
    ERC20MaxVal_Sent=float('-inf') # Maximum value in Ether received from ERC20 token transactions for account
    ERC20AvgVal_Sent=0 # Average value in Ether received from ERC20 token transactions for account

    ERC20MinVal_Rec=float('inf')# Minimum value in Ether sent from ERC20 token transactions for account
    ERC20MaxVal_Rec=float('-inf')# Maximum value in Ether sent from ERC20 token transactions for account
    ERC20AvgVal_Rec=0# Average value in Ether sent from ERC20 token transactions for account

    ERC20UniqSentTokenName=0 # Number of Unique ERC20 tokens transferred
    ERC20UniqRecTokenName=0 # Number of Unique ERC20 tokens received
    ERC20MostSentTokenType=0 # Most sent token for account via ERC20 transaction
    ERC20MostRecTokenType=0 # Most received token for account via ERC20 transactions

   

    tr_list=data.get('result') #Transection List from get request
    st_tr=[] #List of sent transections
    rc_tr=[] #List of received received
    ot_tr=[] #List of other received
    s_adr=set() # set of sent addresses
    r_adr=set() # set of received addresses
    o_adr=set() # set of other addresses(contract address related)
    s_tk=set() # set sent token names 
    r_tk=set() # set received token names

    s_token_type={} # used to track most sent token type
    r_token_type={} # used to track most received token type

    TotalERC20Tnxs=len(tr_list)
    if TotalERC20Tnxs>0:
        for i in tr_list:
            temp_contract=""
            flag=False
            if i['from']==address:
                temp_contract=i['to']
            else:
                temp_contract=i['from']
                
            if temp_contract in s_adr:
                flag=False
            elif temp_contract in r_adr:
                flag=False
            elif temp_contract in o_adr:
                flag=True
            else:
                try:
                    flag=IsCotractAdderss(temp_contract)
                except Exception as e:
                    return e



            if (i['from']==address) & (flag==False):
                st_tr.append(i)
                s_adr.add(i['to'])
                ERC20MinVal_Sent=min(ERC20MinVal_Sent,int(i['value']))
                ERC20MaxVal_Sent=max(ERC20MaxVal_Sent,int(i['value']))
                ERC20TotalEther_Sent+=int(i['value'])
                s_tk.add(i['tokenName'])

                if i['tokenName'] in s_token_type:
                    s_token_type[ i['tokenName'] ] += 1
                else:
                    s_token_type[ i['tokenName'] ] = 1


            elif (i['to']==address) & (flag==False) :
                rc_tr.append(i)
                r_adr.add(i['from'])
                ERC20MinVal_Rec=min(ERC20MinVal_Rec,int(i['value']))
                ERC20MaxVal_Rec=max(ERC20MaxVal_Rec,int(i['value']))
                ERC20TotalEther_Received+=int(i['value'])
                r_tk.add(i['tokenName'])
            
                if i['tokenName'] in r_token_type:
                    r_token_type[ i['tokenName'] ] += 1
                else:
                    r_token_type[ i['tokenName'] ] = 1

            else:
                ot_tr.append(i)
                o_adr.add(temp_contract)
                ERC20TotalEtherSentContract+=int(i['value'])
        if len(s_token_type)!=0:
            ERC20MostSentTokenType = max(s_token_type , key=s_token_type.get)
        if len(r_token_type)!=0:
            ERC20MostRecTokenType = max(r_token_type , key=r_token_type.get)
        st=len(st_tr)
        rt=len(rc_tr)
        ct=len(ot_tr)

        ERC20UniqSent_Addr = len(s_adr)
        ERC20UniqRec_Addr = len(r_adr)
        ERC20UniqRecContractAddr = len(o_adr)

        ERC20UniqRecTokenName=len(r_tk)
        ERC20UniqSentTokenName=len(s_tk)

        if st>1:
            ERC20AvgVal_Sent = ERC20TotalEther_Sent/(st-1)
        if rt>1:
            ERC20AvgVal_Rec = ERC20TotalEther_Received/(rt-1)

        if len(st_tr)>=2:
            for i in range(1,len(st_tr)):
                ERC20AvgTimeBetweenSent_Tnx+= abs( int(st_tr[i]['timeStamp'])-int(st_tr[i-1]['timeStamp']) )
            ERC20AvgTimeBetweenSent_Tnx=(ERC20AvgTimeBetweenSent_Tnx/(st-1))
        if len(rc_tr)>=2:
            for i in range(1,len(rc_tr)):
                ERC20AvgTimeBetweenRec_Tnx+= abs( int(rc_tr[i]['timeStamp'])-int(rc_tr[i-1]['timeStamp']) )
            ERC20AvgTimeBetweenRec_Tnx=(ERC20AvgTimeBetweenRec_Tnx/(rt-1))
        if len(ot_tr)>=2:
            for i in range(1,len(ot_tr)):
                ERC20AvgTimeBetweenContract_Tnx+= abs( int(ot_tr[i]['timeStamp'])-int(ot_tr[i-1]['timeStamp']) )
            ERC20AvgTimeBetweenContract_Tnx=(ERC20AvgTimeBetweenContract_Tnx/(ct-1))
    
    if ERC20MinVal_Sent==float('inf') :
        ERC20MinVal_Sent=0
    if ERC20MaxVal_Sent==float('-inf'):
        ERC20MaxVal_Sent=0
    if ERC20MinVal_Rec==float('inf'):
        ERC20MinVal_Rec=0
    if ERC20MaxVal_Rec==float('-inf'):
        ERC20MaxVal_Rec=0

    
    return {
        "TotalERC20Tnxs:": TotalERC20Tnxs,
        "ERC20TotalEther_Received:": ERC20TotalEther_Received,
        "ERC20TotalEther_Sent:": ERC20TotalEther_Sent,
        "ERC20TotalEtherSentContract:": ERC20TotalEtherSentContract,
        "ERC20UniqSent_Addr:": ERC20UniqSent_Addr,
        "ERC20UniqRec_Addr:": ERC20UniqRec_Addr,
        "ERC20UniqRecContractAddr:": ERC20UniqRecContractAddr,
        "ERC20AvgTimeBetweenSent_Tnx": ERC20AvgTimeBetweenSent_Tnx,
        "ERC20AvgTimeBetweenRec_Tnx": ERC20AvgTimeBetweenRec_Tnx,
        "ERC20AvgTimeBetweenContract_Tnx": ERC20AvgTimeBetweenContract_Tnx,
        "ERC20MinVal_Rec": ERC20MinVal_Rec,
        "ERC20MaxVal_Rec": ERC20MaxVal_Rec,
        "ERC20AvgVal_Rec": ERC20AvgVal_Rec,
        "ERC20MinVal_Sent": ERC20MinVal_Sent,
        "ERC20MaxVal_Sent": ERC20MaxVal_Sent,
        "ERC20AvgVal_Sent": ERC20AvgVal_Sent,
        "ERC20UniqSentTokenName": ERC20UniqSentTokenName,
        "ERC20UniqRecTokenName": ERC20UniqRecTokenName,
        "ERC20MostSentTokenType": ERC20MostSentTokenType,
        "ERC20MostRecTokenType": ERC20MostRecTokenType
    }

def FeatureExtractionFromAddress(Address):
    try:
        EOA_transaction_features = NonmalTransection(Address)
       
    except Exception as e:
        print("NonmalTransection")
        return e
    try:
       contract_transaction_features =  ERC20Transection(Address)
    except Exception as e:
        print("ERC20Transection")
        return e
    return { **EOA_transaction_features, **contract_transaction_features }

In [13]:
import sys
import requests
import logging
import pandas as pd
import numpy as np
from time import sleep

logging.basicConfig(filename="00_log.txt", level=logging.INFO, format="%(message)s")

""" Ignore the next two lines and keep them as they are. """
LINE_UP = "\033[1A"
LINE_CLEAR = "\x1b[2K"


def main():

    df = pd.read_csv("MargedAddresses.csv")

    df.drop(df.columns[0], axis=1, inplace=True)

    print(df.columns)

    new_df = pd.DataFrame()

    """
    Index no. from which the features will start being extracted. Initially set to zero.
    But if program crashes for some reason, then put the index where the program crashed
    as the value of 'START_FROM' variable when you are going to run the program again. 
    """
    START_FROM = 0

    print("Loop starting...")

    for index, address in enumerate(df["Address"].to_list()):

        print(LINE_UP, end=LINE_CLEAR)
        print(f"At index: {index}")

        if index < START_FROM:
            continue
        if index>6:
            break

        try:
            features = FeatureExtractionFromAddress(address)

            row = {"address": address, **features}

            new_df = pd.concat([new_df, pd.DataFrame(row, index=[index])])

            new_df.to_csv("ethereum_scam_dataset.csv")

        except Exception as e:
            print(f"Error: {e.args}")
            sleep(2)
            print(LINE_UP, end=LINE_CLEAR)
            logging.info(f"At index: {index} \t Address: {address}")


if __name__ == "__main__":
    main()


Index(['Address'], dtype='object')
Loop starting...
At index: 0
At index: 1
At index: 2
At index: 3
At index: 4
At index: 5
At index: 6
{'Avg min between sent tnx': 0, 'Avg min between received tnx': 192450.66666666666, 'Time Diff between first and_last (Mins)': 577352, 'Sent_tnx': 0, 'Received_tnx': 4, 'NumberofCreated_Contracts': 0, 'UniqueReceivedFrom_Addresses': 2, 'UniqueSentTo_Addresses20': 0, 'MinValueReceived': 585408480000000000, 'MaxValueReceived': 947515130000000000, 'AvgValueReceived5Average': 2.368210325e+17, 'MinValSent': 0, 'MaxValSent': 0, 'AvgValSent': 0, 'MinValueSentToContract': 0, 'MaxValueSentToContract': 0, 'AvgValueSentToContract': 0, 'TotalTransactions(IncludingTnxtoCreate_Contract)': 4, 'TotalEtherSent': 0, 'TotalEtherReceived': 947284130000000000, 'TotalEtherSent_Contracts': 0, 'TotalEtherBalance': '10000000000000000', 'TotalERC20Tnxs:': 3, 'ERC20TotalEther_Received:': 0, 'ERC20TotalEther_Sent:': 0, 'ERC20TotalEtherSentContract:': 2684000000000000000, 'ERC20Un